In [13]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from array import array
import os
from skimage import io
import cv2

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [14]:
def my_metrics(y_test, y_pred):
    # Evaluation metrics for classification
    confusion = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:")
    print(confusion)
    accuracy = round(accuracy_score(y_test,y_pred),2)
    print("Accuracy: ", accuracy)
    precision = round(precision_score(y_test, y_pred, average = 'weighted'),2)
    print("Precision: ", precision)
    recall = round(recall_score(y_test, y_pred, average = 'weighted'),2)
    print("Recall: ", recall)
    f1 = round(f1_score(y_test, y_pred, average = 'weighted'),2)
    print("F1 score: ", f1)

In [15]:
def fit_clf_and_predict(x,y, test_x, classifier):
    clf = classifier()
    clf.fit(x,y)
    
    y_pred = clf.predict(test_x)
    
    return y_pred

In [17]:
def train_test_and_metrics(train_func, test_func, feat_ext_name, classifier):
    train_root_folder = "./dataset/train"
    test_root_folder = "./dataset/test"   
    
    if feat_ext_name == "Sobel":
        train_data_x, train_data_y, train_data_xy, labels = train_func(train_root_folder)
        
        test_data_x, test_data_y, test_data_xy, filenames = test_func(test_root_folder)
        
        y_pred_x = fit_clf_and_predict(train_data_x, labels, test_data_x, classifier)
        y_pred_y = fit_clf_and_predict(train_data_y, labels, test_data_y, classifier)
        y_pred_xy = fit_clf_and_predict(train_data_xy, labels, test_data_xy, classifier)
        
        print("Metrics for", feat_ext_name, "Feature Extraction:")
        print("Detection on x-axis:")
        my_metrics(filenames, y_pred_x)
        print("Detection on y-axis:")
        my_metrics(filenames, y_pred_y)
        print("Detection on both axes:")
        my_metrics(filenames, y_pred_xy)
        
    else:
        imgs, labels = train_func(train_root_folder)

        t_imgs, t_labels = test_func(test_root_folder)

        y_pred = fit_clf_and_predict(imgs, labels, t_imgs, classifier)
        
        print("Metrics for", feat_ext_name, "Feature Extraction:")
        my_metrics(t_labels, y_pred)

# Decision Trees

## Gabor

In [18]:
# Training data

# Function to load images from folders and assign labels 
def train_images_to_array_gabor(root_folder):
    images = []
    labels = []
    for label in os.listdir(root_folder):
        i = 0
        label_folder = os.path.join(root_folder, label)
        if os.path.isdir(label_folder):
            for filename in os.listdir(label_folder):
                img_path = os.path.join(label_folder, filename)
                try:
                    img = cv2.imread(img_path)
                    if img is not None:
                        print(i, end = '\r')
                        #print((len(os.listdir(root_folder)) - i), end='\r') #Serves as an ad-hoc progress bar for img processing
                        i += 1
                        gabor_features = gabor_texture_analysis(img)
                        images.append(gabor_features) 
                        labels.append(label)
                except:
                    continue
    return np.array(images), np.array(labels)


# Test data

# Function to load images from folders and convert to NumPy array
def test_images_to_array_gabor(root_folder):
    data = []
    filenames = []
    i = 0
    for filename in os.listdir(root_folder):
        img_path = os.path.join(root_folder, filename)
        try:
            img = cv2.imread(img_path)
            if img is not None:
                print(i, end = '\r')
                #print((len(os.listdir(root_folder)) - i), end='\r') #Serves as an ad-hoc progress bar for img processing
                i += 1
                gabor_features = gabor_texture_analysis(img)
                data.append(gabor_features) 
                filenames.append(filename.split()[0])
        except:
            continue
    return np.array(data), filenames


# Gabor Feature Extraction
def gabor_texture_analysis(image, num_theta=8, num_lambda=5):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features = []

    # Define the range for theta (orientation) and lambda (wavelength)
    thetas = np.linspace(0, np.pi, num_theta)
    lambdas = np.linspace(0.1, 0.5, num_lambda)

    # Loop over each theta and lambda
    for theta in thetas:
        for lambd in lambdas:
            # Create Gabor kernel
            kernel = cv2.getGaborKernel((21, 21), 5.0, theta, lambd, 1.0, 0, ktype=cv2.CV_32F)
            # Apply the Gabor kernel to the image
            filtered = cv2.filter2D(gray, cv2.CV_8UC3, kernel)
            # Compute mean and standard deviation of the filtered image
            mean, std_dev = cv2.meanStdDev(filtered)
            # Append mean and standard deviation as features
            features.extend([mean[0][0], std_dev[0][0]])

    return features

In [19]:
train_test_and_metrics(train_images_to_array_gabor, test_images_to_array_gabor, "Gabor", DecisionTreeClassifier)

Metrics for Gabor Feature Extraction:
Confusion Matrix:
[[237   0  18  30  15]
 [  0 295   4   0   1]
 [ 15   3 264   4  14]
 [ 18   0   5 277   0]
 [ 17   1  23   1 258]]
Accuracy:  0.89
Precision:  0.89
Recall:  0.89
F1 score:  0.89


## Sobel

In [20]:
# Training data

# Function to load images from folders and assign labels
def train_images_to_array_sobel(root_folder):
    data_x = []
    data_y = []
    data_xy = []
    labels = []
    for label in os.listdir(root_folder):
        i = 0
        label_folder = os.path.join(root_folder, label)
        if os.path.isdir(label_folder):
            for filename in os.listdir(label_folder):
                img_path = os.path.join(label_folder, filename)
                try:
                    img = cv2.imread(img_path)
                    if img is not None:
                        print(i, end = '\r')
                        #print((len(os.listdir(label_folder)) - i), end='\r') #Serves as an ad-hoc progress bar for img processing
                        i += 1
                        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                        img_blur = cv2.GaussianBlur(img_gray, (3,3), 0)
                        sobel_x, sobel_y, sobelxy = sobel_edge_detection(img_blur, 5)
                        data_x.append(sobel_x.flatten())  # Flatten the image and add to the list
                        data_y.append(sobel_y.flatten())  # Flatten the image and add to the list
                        data_xy.append(sobelxy.flatten())  # Flatten the image and add to the list
                        labels.append(label)  # Assign label from folder name
                except:
                    continue
                    
    return np.array(data_x), np.array(data_y), np.array(data_xy), np.array(labels)


# Test data

# Function to load images from folders and convert to NumPy array
def test_images_to_array_sobel(root_folder):
    data_x = []
    data_y = []
    data_xy = []
    filenames = []
    i = 0
    for filename in os.listdir(root_folder):
        img_path = os.path.join(root_folder, filename)
        try:
            img = cv2.imread(img_path)
            if img is not None:
                print((len(os.listdir(root_folder)) - i), end='\r') #Serves as an ad-hoc progress bar for img processing
                i += 1
                img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img_blur = cv2.GaussianBlur(img_gray, (3,3), 0)
                sobel_x, sobel_y, sobelxy = sobel_edge_detection(img_blur, 5)
                data_x.append(sobel_x.flatten())  # Flatten the image and add to the list
                data_y.append(sobel_y.flatten())  # Flatten the image and add to the list
                data_xy.append(sobelxy.flatten())  # Flatten the image and add to the list
                filenames.append(filename.split()[0])
        except:
            continue
            
    return np.array(data_x), np.array(data_y), np.array(data_xy), filenames


# Sobel Edge Detection
def sobel_edge_detection(image_blur, k_size):
    sobelx = cv2.Sobel(src=image_blur, ddepth=cv2.CV_64F, dx=1, dy=0, ksize= k_size)
    sobely = cv2.Sobel(src=image_blur, ddepth=cv2.CV_64F, dx=0, dy=1, ksize= k_size)
    sobelxy = cv2.Sobel(src=image_blur, ddepth=cv2.CV_64F, dx=1,dy=1, ksize= k_size)
    
    return sobelx, sobely, sobelxy

In [21]:
train_test_and_metrics(train_images_to_array_sobel, test_images_to_array_sobel, "Sobel", DecisionTreeClassifier)

Metrics for Sobel Feature Extraction:
Detection on x-axis:
Confusion Matrix:
[[206   9  23  58   4]
 [  4 267  12   4  13]
 [ 20   8 235   6  31]
 [ 21   4   3 270   2]
 [  3   5  38   0 254]]
Accuracy:  0.82
Precision:  0.82
Recall:  0.82
F1 score:  0.82
Detection on y-axis:
Confusion Matrix:
[[225   6  18  48   3]
 [  6 265  14   4  11]
 [ 19  11 230   3  37]
 [ 46   1   5 246   2]
 [  5   9  51   2 233]]
Accuracy:  0.8
Precision:  0.8
Recall:  0.8
F1 score:  0.8
Detection on both axes:
Confusion Matrix:
[[225   4  32  33   6]
 [ 10 253  15   2  20]
 [ 32   7 208   7  46]
 [ 38   0   3 257   2]
 [ 10  15  49   1 225]]
Accuracy:  0.78
Precision:  0.78
Recall:  0.78
F1 score:  0.78


## Canny

In [22]:
# Training data

# Function to load images from folders and assign labels 
def train_images_to_array_canny(root_folder):
    images = []
    labels = []
    for label in os.listdir(root_folder):
        i = 0
        label_folder = os.path.join(root_folder, label)
        if os.path.isdir(label_folder):
            for filename in os.listdir(label_folder):
                img_path = os.path.join(label_folder, filename)
                try:
                    img = cv2.imread(img_path)
                    if img is not None:
                        print((len(os.listdir(label_folder)) - i), end='\r') #Serves as an ad-hoc progress bar for img processing
                        i += 1
                        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                        img_blur = cv2.GaussianBlur(img_gray, (3,3), 0)
                        edges = cv2.Canny(image=img_blur, threshold1=100, threshold2=200)
                        images.append(edges.flatten())  # Flatten the image and add to the list
                        labels.append(label)  # Assign label from folder name
                except:
                    continue
                    
    return np.array(images), np.array(labels)


# Test data

# Function to load images from folders and convert to NumPy array
def test_images_to_array_canny(root_folder):
    data = []
    filenames = []
    i = 0
    for filename in os.listdir(root_folder):
        img_path = os.path.join(root_folder, filename)
        try:
            img = cv2.imread(img_path)
            if img is not None:
                print((len(os.listdir(root_folder)) - i), end='\r') #Serves as an ad-hoc progress bar for img processing
                i += 1
                img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img_blur = cv2.GaussianBlur(img_gray, (3,3), 0)
                edges = cv2.Canny(image=img_blur, threshold1=100, threshold2=200)
                data.append(edges.flatten())  # Append hog image
                filenames.append(filename.split()[0])
        except:
            continue
            
    return np.array(data), filenames


# Canny Edge Detection
def canny_edge_detection(image_blur, threshold_1, threshold_2):
    edges = cv2.Canny(image=img_blur, threshold1=threshold_1, threshold2=threshold_2)
    
    return edges


In [23]:
train_test_and_metrics(train_images_to_array_canny, test_images_to_array_canny, "Canny", DecisionTreeClassifier)

Metrics for Canny Feature Extraction:
Confusion Matrix:
[[147  14  29  99  11]
 [ 14 248   1  34   3]
 [ 39  13 133  20  95]
 [ 94  21   5 178   2]
 [ 23   5 110  10 152]]
Accuracy:  0.57
Precision:  0.57
Recall:  0.57
F1 score:  0.57


## Hog

In [24]:
from skimage.feature import hog
import matplotlib.image as mpimg

# Training data

# Function to load images from folders, assign labels, and convert to HOG
def train_images_to_array_hog(root_folder):
    images = []
    labels = []
    for label in os.listdir(root_folder):
        i = 0
        print(label)
        label_folder = os.path.join(root_folder, label)
        if os.path.isdir(label_folder):
            for filename in os.listdir(label_folder):
                img_path = os.path.join(label_folder, filename)
                try:
                    img = Image.open(img_path).convert("L")  # Convert image to grayscale
                    if img is not None:
                        print((len(os.listdir(label_folder)) - i), end='\r') #Serves as ad-hoc progress bar for img processing
                        i += 1
                        img_array = np.array(img)
                        img = hog(img_array)
                        images.append(img)  # Flatten the HOG data and add to the list
                        labels.append(label)  # Assign label from folder name
                except:
                    continue
    return np.array(images), np.array(labels)


# Test data

# Function to load images from folders and convert to HOG
def test_images_to_array_hog(root_folder):
    data = []
    filenames = []
    i = 0
    for filename in os.listdir(root_folder):
        img_path = os.path.join(root_folder, filename)
        try:
            img = Image.open(img_path).convert("L")  # Convert image to grayscale
            if img is not None:
                print((len(os.listdir(root_folder)) - i), end='\r') #Serves as an ad-hoc progress bar for img processing
                i += 1
                img_array = np.array(img)
                img = hog(img_array)
                data.append(img)  # Append HOG data
                filenames.append(filename.split()[0])
        except:
            continue
    return np.array(data), filenames

In [25]:
train_test_and_metrics(train_images_to_array_hog, test_images_to_array_hog, "Hog", DecisionTreeClassifier)

.DS_Store
Arborio
basmati
Ipsala
Jasmine
Karacadag
Metrics for Hog Feature Extraction:
Confusion Matrix:
[[258   2  16  23   1]
 [  4 291   0   0   5]
 [ 10   4 265   0  21]
 [ 19   0   0 280   1]
 [  2   0  23   0 275]]
Accuracy:  0.91
Precision:  0.91
Recall:  0.91
F1 score:  0.91


# Neural Network

In [26]:
import splitfolders
from tensorflow.keras.preprocessing import image_dataset_from_directory # type: ignore
import numpy as np
import matplotlib.pyplot as plt # type: ignore
import seaborn as sns # type: ignore
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout # type: ignore
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [27]:
# Load data
train_dataset = image_dataset_from_directory(
 "ricedata/train",
    labels = 'inferred',
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
   "ricedata/test",
    labels = 'inferred',
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
   "ricedata/val",
    labels = 'inferred',
    image_size=(180, 180),
    batch_size=32)

Found 42000 files belonging to 5 classes.
Found 6000 files belonging to 5 classes.
Found 12000 files belonging to 5 classes.


## Gabor Application

In [28]:
# Gabor
def build_gabor_kernel(ksize, sigma, theta, lambd, gamma, psi):
    # Define the size of the filter
    axis = tf.range(-ksize // 2 + 1, ksize // 2 + 1, dtype=tf.float32)
    x, y = tf.meshgrid(axis, axis)

    # Rotation parameters
    x_theta = x * tf.math.cos(theta) + y * tf.math.sin(theta)
    y_theta = -x * tf.math.sin(theta) + y * tf.math.cos(theta)

    gb = tf.exp(-0.5 * (x_theta**2 + gamma**2 * y_theta**2) / sigma**2) * tf.cos(2. * np.pi * x_theta / lambd + psi)
    gb = tf.expand_dims(gb, -1)  # For grayscale
    gb = tf.expand_dims(gb, -1)  # For filter
    return gb

def apply_gabor_filter(image, ksize=21, sigma=8, theta=1.57, lambd=10, gamma=0.5, psi=0):
    kernel = build_gabor_kernel(ksize, sigma, theta, lambd, gamma, psi)
    # Convert image to grayscale and add batch and channel dimensions
    image = tf.image.rgb_to_grayscale(image)
    filtered_image = tf.nn.conv2d(image, kernel, strides=[1, 1, 1, 1], padding="SAME")
    return filtered_image

def preprocess_image(image, label):
    # Apply the Gabor filter
    filtered_image = apply_gabor_filter(image)
    return filtered_image, label

train_dataset_gabor = train_dataset.map(preprocess_image)
validation_dataset_gabor = validation_dataset.map(preprocess_image)
test_dataset_gabor = test_dataset.map(preprocess_image)

## HOG Application

In [ ]:
# HOG
def approximate_hog(image):
    # Convert to grayscale
    image = tf.image.rgb_to_grayscale(image)
    # Calculate gradients
    dx, dy = tf.image.image_gradients(image)
    # Compute gradient magnitude and angle
    magnitude = tf.sqrt(tf.square(dx) + tf.square(dy))
    angle = tf.atan2(dy, dx)
    # Further processing can be done to mimic histogramming of gradients in cells as done in HOG
    return magnitude  # Simplified output for this example

def preprocess_image(image, label):
    # Apply approximate HOG
    image = approximate_hog(image)
    return image, label

# Update your dataset processing
train_dataset_hog = train_dataset.map(preprocess_image)
validation_dataset_hog = validation_dataset.map(preprocess_image)
test_dataset_hog = test_dataset.map(preprocess_image)

## Sobel Application

In [ ]:
# Sobel
def apply_sobel(image):
    # Sobel filter kernels for edge detection in x and y directions
    sobel_x = tf.constant([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=tf.float32)
    sobel_y = tf.constant([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=tf.float32)
    sobel_x_filter = tf.reshape(sobel_x, [3, 3, 1, 1])
    sobel_y_filter = tf.reshape(sobel_y, [3, 3, 1, 1])

    # Convert image to grayscale
    gray = tf.image.rgb_to_grayscale(image)

    # Expand dimensions to apply the filter
    gray = tf.expand_dims(gray, axis=0)

    # Apply Sobel filtering
    filtered_x = tf.nn.conv2d(input=gray, filters=sobel_x_filter, strides=[1, 1, 1, 1], padding='SAME')
    filtered_y = tf.nn.conv2d(input=gray, filters=sobel_y_filter, strides=[1, 1, 1, 1], padding='SAME')

    # Calculate the magnitude of the gradients
    sobel_output = tf.sqrt(tf.square(filtered_x) + tf.square(filtered_y))

    # Remove dimensions of size 1 from the shape of the tensor
    sobel_output = tf.squeeze(sobel_output, axis=0)

    return sobel_output

def preprocess_image(image, label):
    # Apply the Sobel filter
    image = apply_sobel(image)
    return image, label

train_dataset_sobel = train_dataset.map(preprocess_image)
validation_dataset_sobel = validation_dataset.map(preprocess_image)
test_dataset_sobel = test_dataset.map(preprocess_image)

## Model Building

In [ ]:
# Build the model

def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 1)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(5, activation='softmax')
    ])
    return model

model = build_model()

## Gabor Predictions

In [29]:
# Gabor
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(train_dataset_gabor, epochs=1, validation_data=validation_dataset_gabor)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset_gabor)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Confusion matrix and additional metrics
true_labels = []
predictions = []

for images, labels in test_dataset_gabor:
    pred = model.predict(images)  # Use the preprocessed test dataset
    true_labels.extend(labels.numpy())
    predictions.extend(np.argmax(pred, axis=1))

# Compute the confusion matrix
cm = confusion_matrix(true_labels, predictions)

# Calculate precision, recall, and F1-score
precision = precision_score(true_labels, predictions, average='macro')
recall = recall_score(true_labels, predictions, average='macro')
f1 = f1_score(true_labels, predictions, average='macro')

# Print metrics
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Plotting the confusion matrix
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

Epoch 1/10
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 466s 353ms/step - accuracy: 0.8626 - loss: 16.9477 - val_accuracy: 0.9592 - val_loss: 0.1148
Epoch 2/10
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 460s 350ms/step - accuracy: 0.9519 - loss: 0.1452 - val_accuracy: 0.9694 - val_loss: 0.0905
Epoch 3/10
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 524s 399ms/step - accuracy: 0.9605 - loss: 0.1252 - val_accuracy: 0.9687 - val_loss: 0.1034
Epoch 4/10
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 460s 350ms/step - accuracy: 0.9613 - loss: 0.1227 - val_accuracy: 0.9721 - val_loss: 0.0882
Epoch 5/10
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 463s 352ms/step - accuracy: 0.9683 - loss: 0.0998 - val_accuracy: 0.9716 - val_loss: 0.0851
Epoch 6/10
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 461s 350ms/step - accuracy: 0.9674 - loss: 0.0959 - val_accuracy: 0.9708 - val_loss: 0.0850
Epoch 7/10
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 462s 351ms/step - accuracy: 0.9667 - loss: 0.1031 - val_accuracy: 0.9722 - val_loss: 0.0825
Epoch 8/10
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 469s 357ms/step - a

## HOG Predictions

In [ ]:
# HOG
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(train_dataset_hog, epochs=10, validation_data=validation_dataset_hog)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset_hog)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Confusion matrix and additional metrics
true_labels = []
predictions = []

for images, labels in test_dataset_hog:
    pred = model.predict(images)  # Use the preprocessed test dataset
    true_labels.extend(labels.numpy())
    predictions.extend(np.argmax(pred, axis=1))

# Compute the confusion matrix
cm = confusion_matrix(true_labels, predictions)

# Calculate precision, recall, and F1-score
precision = precision_score(true_labels, predictions, average='macro')
recall = recall_score(true_labels, predictions, average='macro')
f1 = f1_score(true_labels, predictions, average='macro')

# Print metrics
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Plotting the confusion matrix
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

## Sobel Predictions

In [ ]:
# Sobel
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(train_dataset_sobel, epochs=10, validation_data=validation_dataset_sobel)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset_sobel)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Confusion matrix and additional metrics
true_labels = []
predictions = []

for images, labels in test_dataset_sobel:
    pred = model.predict(images)  # Use the preprocessed test dataset
    true_labels.extend(labels.numpy())
    predictions.extend(np.argmax(pred, axis=1))

# Compute the confusion matrix
cm = confusion_matrix(true_labels, predictions)

# Calculate precision, recall, and F1-score
precision = precision_score(true_labels, predictions, average='macro')
recall = recall_score(true_labels, predictions, average='macro')
f1 = f1_score(true_labels, predictions, average='macro')

# Print metrics
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Plotting the confusion matrix
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


# Random Forest

## Gabor

In [31]:
train_test_and_metrics(train_images_to_array_gabor, test_images_to_array_gabor, "Gabor", RandomForestClassifier)

Metrics for Gabor Feature Extraction:
Confusion Matrix:
[[276   0  13   9   2]
 [  1 295   4   0   0]
 [  2   0 290   0   8]
 [  5   0   1 294   0]
 [  0   0  11   0 289]]
Accuracy:  0.96
Precision:  0.96
Recall:  0.96
F1 score:  0.96


## Sobel

In [32]:
train_test_and_metrics(train_images_to_array_sobel, test_images_to_array_sobel, "Sobel", RandomForestClassifier)

Metrics for Sobel Feature Extraction:
Detection on x-axis:
Confusion Matrix:
[[281   0   5  14   0]
 [  1 299   0   0   0]
 [  1   2 281   0  16]
 [ 10   0   0 290   0]
 [  1   0   7   0 292]]
Accuracy:  0.96
Precision:  0.96
Recall:  0.96
F1 score:  0.96
Detection on y-axis:
Confusion Matrix:
[[280   0   4  16   0]
 [  1 299   0   0   0]
 [  0   8 272   0  20]
 [ 10   0   0 290   0]
 [  0   0   8   0 292]]
Accuracy:  0.96
Precision:  0.96
Recall:  0.96
F1 score:  0.96
Detection on both axes:
Confusion Matrix:
[[283   0   4  13   0]
 [  1 299   0   0   0]
 [  1   9 251   0  39]
 [ 12   0   0 288   0]
 [  1   0   6   0 293]]
Accuracy:  0.94
Precision:  0.94
Recall:  0.94
F1 score:  0.94


## Canny

In [33]:
train_test_and_metrics(train_images_to_array_canny, test_images_to_array_canny, "Canny", RandomForestClassifier)

Metrics for Canny Feature Extraction:
Confusion Matrix:
[[202   2   7  89   0]
 [ 14 266   1  19   0]
 [ 25   0 195   1  79]
 [ 41   8   0 251   0]
 [  3   0 111   0 186]]
Accuracy:  0.73
Precision:  0.74
Recall:  0.73
F1 score:  0.73


## HOG

In [34]:
train_test_and_metrics(train_images_to_array_hog, test_images_to_array_hog, "Hog", RandomForestClassifier)

.DS_Store
Arborio
basmati
Ipsala
Jasmine
Karacadag
Metrics for Hog Feature Extraction:
Confusion Matrix:
[[277   0   7  16   0]
 [  1 299   0   0   0]
 [  0   3 285   0  12]
 [ 13   0   0 287   0]
 [  0   0   5   0 295]]
Accuracy:  0.96
Precision:  0.96
Recall:  0.96
F1 score:  0.96


# Support Vector Machine (SVM)

In [41]:
def svm_metrics(train_func, test_func):
    images, labels = train_func("./dataset/train")
    t_imgs, t_labels = test_func("./dataset/test")
    
    x_train = images
    y_train = labels
    
    x_test = t_imgs
    y_test = t_labels
    
    y_pred_poly_full = svm_algorithm(x_train, y_train, x_test, 'poly', 5 )
    print('Full Dataset SVM Kernel= Poly')
    my_metrics(y_test, y_pred_poly_full)

    y_pred_rbf_full = svm_algorithm(x_train, y_train, x_test, 'rbf', 5 )
    print('Full Dataset SVM Kernel= Radias Bias Function')
    my_metrics(y_test, y_pred_rbf_full)

    # Kernel is Linear
    y_pred_linear_full = svm_algorithm(x_train, y_train, x_test, 'linear', 5)
    print('Full Dataset SVM Kernel= Linear')
    my_metrics(y_test, y_pred_linear_full)

    y_pred_sig_full = svm_algorithm(x_train, y_train, x_test, 'sigmoid', 5 )
    print('Full Dataset SVM Kernel= Sigmoid')
    my_metrics(y_test, y_pred_sig_full)

## Gabor

In [35]:
from sklearn import svm

In [36]:
def svm_algorithm(x_train, y_train, x_test, kernel_type, c_value):
    # Create SVM classifier
    svm_classifier = svm.SVC(kernel=kernel_type, C=c_value)

    # Train the classifier
    svm_classifier.fit(x_train, y_train)

    # Predict on the test data
    y_pred = svm_classifier.predict(x_test)

    return y_pred

In [37]:
images, labels = train_images_to_array_gabor("./dataset/train")
t_imgs, t_labels = test_images_to_array_gabor("./dataset/test")

In [38]:
x_train = images
y_train = labels
x_test = t_imgs
y_test = t_labels

In [39]:
y_pred_poly_full = svm_algorithm(x_train, y_train, x_test, 'poly', 5 )
print('Full Dataset SVM Kernel= Poly')
my_metrics(y_test, y_pred_poly_full)

y_pred_rbf_full = svm_algorithm(x_train, y_train, x_test, 'rbf', 5 )
print('Full Dataset SVM Kernel= Radias Bias Function')
my_metrics(y_test, y_pred_rbf_full)

# Kernel is Linear
y_pred_linear_full = svm_algorithm(x_train, y_train, x_test, 'linear', 5)
print('Full Dataset SVM Kernel= Linear')
my_metrics(y_test, y_pred_linear_full)

y_pred_sig_full = svm_algorithm(x_train, y_train, x_test, 'sigmoid', 5 )
print('Full Dataset SVM Kernel= Sigmoid')
my_metrics(y_test, y_pred_sig_full)

Full Dataset SVM Kernel= Poly
Confusion Matrix:
[[272   0  15   9   4]
 [  1 299   0   0   0]
 [  6  13 242   1  38]
 [  5   0   3 292   0]
 [ 11   1  20   0 268]]
Accuracy:  0.92
Precision:  0.91
Recall:  0.92
F1 score:  0.91
Full Dataset SVM Kernel= Radias Bias Function
Confusion Matrix:
[[270   0  14  10   6]
 [  1 299   0   0   0]
 [  6  13 234   1  46]
 [  6   0   2 292   0]
 [  9   1  19   0 271]]
Accuracy:  0.91
Precision:  0.91
Recall:  0.91
F1 score:  0.91
Full Dataset SVM Kernel= Linear
Confusion Matrix:
[[294   0   4   2   0]
 [  1 298   1   0   0]
 [  6   3 282   0   9]
 [  4   0   0 296   0]
 [  4   0  23   0 273]]
Accuracy:  0.96
Precision:  0.96
Recall:  0.96
F1 score:  0.96
Full Dataset SVM Kernel= Sigmoid
Confusion Matrix:
[[  0 300   0   0   0]
 [  0   2 298   0   0]
 [  0 274  26   0   0]
 [  0 300   0   0   0]
 [  0 287  13   0   0]]
Accuracy:  0.02
Precision:  0.02
Recall:  0.02
F1 score:  0.02


## HOG

In [42]:
svm_metrics(train_images_to_array_hog, test_images_to_array_hog)

.DS_Store
Arborio
basmati
Ipsala
Jasmine
Karacadag
Full Dataset SVM Kernel= Poly
Confusion Matrix:
[[290   0   1   9   0]
 [  1 297   2   0   0]
 [  0   0 297   0   3]
 [  4   0   0 296   0]
 [  0   0   6   0 294]]
Accuracy:  0.98
Precision:  0.98
Recall:  0.98
F1 score:  0.98
Full Dataset SVM Kernel= Radias Bias Function
Confusion Matrix:
[[290   0   1   9   0]
 [  1 297   2   0   0]
 [  0   0 296   0   4]
 [  4   0   0 296   0]
 [  0   0   6   0 294]]
Accuracy:  0.98
Precision:  0.98
Recall:  0.98
F1 score:  0.98
Full Dataset SVM Kernel= Linear
Confusion Matrix:
[[289   0   1  10   0]
 [  2 298   0   0   0]
 [  0   0 293   0   7]
 [  4   0   0 296   0]
 [  0   0   6   0 294]]
Accuracy:  0.98
Precision:  0.98
Recall:  0.98
F1 score:  0.98
Full Dataset SVM Kernel= Sigmoid
Confusion Matrix:
[[284   0   2  14   0]
 [  2 298   0   0   0]
 [  0   0 293   0   7]
 [  4   0   0 296   0]
 [  0   0   6   0 294]]
Accuracy:  0.98
Precision:  0.98
Recall:  0.98
F1 score:  0.98


## Canny

In [43]:
svm_metrics(train_images_to_array_canny, test_images_to_array_canny)

Full Dataset SVM Kernel= Poly
Confusion Matrix:
[[130   0   0 170   0]
 [  0   0   0 300   0]
 [  0   0 249  51   0]
 [  0   0   0 300   0]
 [  0   0 208  92   0]]
Accuracy:  0.45
Precision:  0.37
Recall:  0.45
F1 score:  0.35
Full Dataset SVM Kernel= Radias Bias Function
Confusion Matrix:
[[278   0   7  15   0]
 [  2 298   0   0   0]
 [  0   3 256   0  41]
 [  7   0   0 293   0]
 [  0   0  13   0 287]]
Accuracy:  0.94
Precision:  0.94
Recall:  0.94
F1 score:  0.94
Full Dataset SVM Kernel= Linear
Confusion Matrix:
[[276   0   3  21   0]
 [  7 293   0   0   0]
 [  3   2 273   0  22]
 [ 12   0   0 288   0]
 [  0   0  16   0 284]]
Accuracy:  0.94
Precision:  0.94
Recall:  0.94
F1 score:  0.94
Full Dataset SVM Kernel= Sigmoid
Confusion Matrix:
[[271   1   4  24   0]
 [  6 294   0   0   0]
 [  2   2 273   0  23]
 [ 15   0   0 285   0]
 [  0   0  22   0 278]]
Accuracy:  0.93
Precision:  0.93
Recall:  0.93
F1 score:  0.93


## Sobel

In [44]:
train_data_x, train_data_y, train_data_xy, labels_sobel = train_images_to_array_sobel("./dataset/train")

test_data_x, test_data_y, test_data_xy, filenames_sobel = test_images_to_array_sobel("./dataset/test")

In [47]:
x_train_sobel_x = np.array(train_data_x)
x_train_sobel_y = np.array(train_data_y)
x_train_sobel_xy = np.array(train_data_xy)
y_train = np.array(labels_sobel)

y_test = np.array(filenames_sobel)

In [ ]:
y_pred_poly_sobel = svm_algorithm(x_train_sobel_x, y_train, test_data_x, 'poly', 5)
print('Dataset SVM Kernel = Poly ')
my_metrics(y_test, y_pred_poly_sobel)

y_pred_poly_sobel = svm_algorithm(x_train_sobel_x, y_train, test_data_x, 'Linear', 5)
print('Dataset SVM Kernel = Linear ')
my_metrics(y_test, y_pred_poly_sobel)

y_pred_poly_sobel = svm_algorithm(x_train_sobel_x, y_train, test_data_x, 'rbf', 5)
print('Dataset SVM Kernel = Radial Bias Function ')
my_metrics(y_test, y_pred_poly_sobel)

y_pred_poly_sobel = svm_algorithm(x_train_sobel_x, y_train, test_data_x, 'sigmoid', 5)
print('Dataset SVM Kernel = sigmoid ')
my_metrics(y_test, y_pred_poly_sobel)